In [571]:
import json
import requests
import numpy as np
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [572]:
us_state_abbrev = {
    'United States': 'USA',
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York City':"NYC",
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [573]:
base_url = "https://data.cdc.gov/resource/9bhg-hcku.json"
response = requests.get(base_url)
cdc_data = response.json()

In [574]:
client = Socrata("data.cdc.gov", None)

results = client.get("9bhg-hcku", limit=6000)

results_df = pd.DataFrame.from_records(results)
#results_df

In [575]:
results_df = results_df.loc[results_df['group']=="By Total"]

In [576]:
sex_df = results_df.loc[results_df['age_group']=="All Ages"]

In [577]:
del sex_df['data_as_of']
del sex_df['start_date']
del sex_df['end_date']
del sex_df['group']
del sex_df['age_group']
del sex_df['total_deaths']
del sex_df['pneumonia_deaths']
del sex_df['pneumonia_and_covid_19_deaths']
del sex_df['influenza_deaths']
del sex_df['pneumonia_influenza_or_covid']
del sex_df['footnote']

In [578]:
male_df = sex_df.loc[sex_df["sex"]=="Male"]
#male_df

In [579]:
female_df = sex_df.loc[sex_df["sex"]=="Female"]
#female_df

In [580]:
merged_df = pd.merge(male_df, female_df, on='state')

In [581]:
#merged_df

In [582]:
all_df = sex_df.loc[sex_df["sex"]=="All Sexes"]
#all_df.head()

In [583]:
new_df = pd.merge(merged_df, all_df, how='left', on='state')

In [584]:
#new_df.head()

In [585]:
del new_df['sex_x']
del new_df['sex_y']
del new_df['sex']
del new_df['year']
del new_df['year_x']
del new_df['year_y']

In [586]:
new_df = new_df.rename(columns={"state":"State", "covid_19_deaths_x":"Male COVID-19 Deaths", "covid_19_deaths_y": "Female COVID-19 Deaths", "covid_19_deaths":"All Sexes COVID-19 Deaths"})

In [587]:
new_df = new_df.loc[new_df['State']!="Puerto Rico"]

In [588]:
#new_df

In [589]:
client = Socrata("data.cdc.gov", None)

results = client.get("9bhg-hcku", limit=6000)

results_df = pd.DataFrame.from_records(results)


In [590]:
results_df = results_df.loc[results_df['group']=="By Total"]

In [591]:
age_df = results_df.loc[results_df['sex']=="All Sexes"]

In [592]:
del age_df['data_as_of']
del age_df['start_date']
del age_df['end_date']
del age_df['group']
del age_df['sex']
del age_df['total_deaths']
del age_df['pneumonia_deaths']
del age_df['pneumonia_and_covid_19_deaths']
del age_df['influenza_deaths']
del age_df['pneumonia_influenza_or_covid']
del age_df['footnote']
del age_df['year']

In [593]:
#age_df

In [594]:
age_df["age_group"].value_counts()

85 years and over    54
25-34 years          54
40-49 years          54
55-64 years          54
0-17 years           54
All Ages             54
30-39 years          54
75-84 years          54
45-54 years          54
1-4 years            54
5-14 years           54
15-24 years          54
18-29 years          54
Under 1 year         54
65-74 years          54
35-44 years          54
50-64 years          54
Name: age_group, dtype: int64

In [595]:
age_df = age_df.loc[age_df['age_group'] != "18-29 years"]
age_df = age_df.loc[age_df['age_group'] != "0-17 years"]
age_df = age_df.loc[age_df['age_group'] != "Under 1 year"]
age_df = age_df.loc[age_df['age_group'] != "30-39 years"]
age_df = age_df.loc[age_df['age_group'] != "40-49 years"]
age_df = age_df.loc[age_df['age_group'] != "50-64 years"]

In [596]:
age_df = age_df.replace({"1-4 years":"Under 5 years"})
age_df['age_group'].value_counts()

85 years and over    54
25-34 years          54
All Ages             54
55-64 years          54
35-44 years          54
65-74 years          54
45-54 years          54
15-24 years          54
5-14 years           54
Under 5 years        54
75-84 years          54
Name: age_group, dtype: int64

In [597]:
#age_df.fillna(0)

In [598]:
under5_df = age_df.loc[age_df["age_group"]=="Under 5 years"]

In [599]:
year5_14_df = age_df.loc[age_df["age_group"]=="5-14 years"]

In [600]:
year15_24_df = age_df.loc[age_df["age_group"]=="15-24 years"]

In [601]:
year25_34_df = age_df.loc[age_df["age_group"]=="25-34 years"]

In [602]:
year35_44_df = age_df.loc[age_df["age_group"]=="35-44 years"]

In [603]:
year45_54_df = age_df.loc[age_df["age_group"]=="45-54 years"]

In [604]:
year55_64_df = age_df.loc[age_df["age_group"]=="55-64 years"]

In [605]:
year65_74_df = age_df.loc[age_df["age_group"]=="65-74 years"]

In [606]:
year75_84_df = age_df.loc[age_df["age_group"]=="75-84 years"]

In [607]:
year85_df = age_df.loc[age_df["age_group"]=="85 years and over"]

In [608]:
all_ages_df = age_df.loc[age_df["age_group"]=="All Ages"]

In [609]:
merged_age = pd.merge(under5_df, year5_14_df, on='state')
merged_age = pd.merge(merged_age, year15_24_df, on ='state')
#merged_age.head()

In [610]:
merged_age = merged_age.rename(columns={
                                "covid_19_deaths_x":"Under 5 Years COVID-19 Deaths", 
                                "covid_19_deaths_y": "5-14 Years COVID-19 Deaths", 
                                "covid_19_deaths":"15-24 Years COVID-19 Deaths"
                                       })
del merged_age['age_group_x']
del merged_age['age_group_y']
del merged_age['age_group']
#merged_age.head()

In [611]:
merged_age = pd.merge(merged_age, year25_34_df, on ='state')
merged_age = pd.merge(merged_age, year35_44_df, on ='state')
merged_age = merged_age.rename(columns={
                                "covid_19_deaths_x":"25-34 Years COVID-19 Deaths", 
                                "covid_19_deaths_y": "35-44 Years COVID-19 Deaths"
                                       })
del merged_age['age_group_x']
del merged_age['age_group_y']
#merged_age.head()

In [612]:
merged_age = pd.merge(merged_age, year45_54_df, on ='state')
merged_age = pd.merge(merged_age, year55_64_df, on ='state')
merged_age = merged_age.rename(columns={
                                "covid_19_deaths_x":"45-54 Years COVID-19 Deaths", 
                                "covid_19_deaths_y": "55-64 Years COVID-19 Deaths"
                                       })
del merged_age['age_group_x']
del merged_age['age_group_y']
#merged_age.head()

In [613]:
merged_age = pd.merge(merged_age, year65_74_df, on ='state')
merged_age = pd.merge(merged_age, year75_84_df, on ='state')
merged_age = merged_age.rename(columns={
                                "covid_19_deaths_x":"65-74 Years COVID-19 Deaths", 
                                "covid_19_deaths_y": "75-84 Years COVID-19 Deaths"
                                       })
del merged_age['age_group_x']
del merged_age['age_group_y']
#merged_age.head()

In [614]:
merged_age = pd.merge(merged_age, year85_df, on ='state')
merged_age = pd.merge(merged_age, all_ages_df, on='state')
merged_age = merged_age.rename(columns={
                                "covid_19_deaths_x":"85 Years and Over COVID-19 Deaths",
                                "covid_19_deaths_y":"All Ages COVID-19 Deaths"
                                       })
del merged_age['age_group_x']
del merged_age['age_group_y']
#merged_age.head()

In [615]:
merged_age = merged_age.fillna(0)

In [616]:
merged_age = merged_age.loc[merged_age['state']!='Puerto Rico']

In [617]:
merged_age = merged_age.rename(columns={"state":"State"})
#merged_age.head()

In [618]:
age_sex_df = pd.merge(new_df, merged_age, on='State')
#age_sex_df

In [619]:
age_sex_df = age_sex_df.fillna(0)
#age_sex_df

In [620]:
for index, row in age_sex_df.iterrows():
    state = row[0]
    age_sex_df.loc[index,"State"] = us_state_abbrev[state]

In [621]:
#age_sex_df

In [622]:
#age_sex_df.to_csv("../data-file/total-age-sex-data.csv", index=False, header=True)

In [623]:
population_df = pd.read_csv("../jennie-folder/Population_byCDCAgeGroup_byState.csv")

In [624]:
for index, row in population_df.iterrows():
    state = row[2]
    population_df.loc[index,"State"] = us_state_abbrev[state]

In [625]:
population_df = population_df[['State', 'CDC_AgeGroups', 'Population']]

In [626]:

under5_pop = population_df.loc[population_df['CDC_AgeGroups']=="Under 5 years"]
years5_14_pop = population_df.loc[population_df['CDC_AgeGroups']=="5-14 years"]
years15_24_pop = population_df.loc[population_df['CDC_AgeGroups']=="15-24 years"]
years25_34_pop = population_df.loc[population_df['CDC_AgeGroups']=="25-34 years"]
years35_44_pop = population_df.loc[population_df['CDC_AgeGroups']=="35-44 years"]
years45_54_pop = population_df.loc[population_df['CDC_AgeGroups']=="45-54 years"]
years55_64_pop = population_df.loc[population_df['CDC_AgeGroups']=="55-64 years"]
years65_74_pop = population_df.loc[population_df['CDC_AgeGroups']=="65-74 years"]
years75_84_pop = population_df.loc[population_df['CDC_AgeGroups']=="75-84 years"]
years85_pop = population_df.loc[population_df['CDC_AgeGroups']=="85 years and over"]
all_ages_pop = population_df.loc[population_df['CDC_AgeGroups']=="Total"]



In [627]:
merged_pop = pd.merge(under5_pop, years5_14_pop, on ='State')
merged_pop = pd.merge(merged_pop, years15_24_pop, on='State')
merged_pop = merged_pop.rename(columns={
                                "Population_x":"Under 5 Years Population",
                                "Population_y":"5-14 Years Population",
                                "Population":"15-24 Years Population"
                                       })
del merged_pop['CDC_AgeGroups_x']
del merged_pop['CDC_AgeGroups_y']
del merged_pop['CDC_AgeGroups']
#merged_pop.head()

In [628]:
merged_pop = pd.merge(merged_pop, years25_34_pop, on ='State')
merged_pop = pd.merge(merged_pop, years35_44_pop, on='State')
merged_pop = pd.merge(merged_pop, years45_54_pop, on='State')
merged_pop = merged_pop.rename(columns={
                                "Population_x":"25-34 Years Population",
                                "Population_y":"35-44 Years Population",
                                "Population":"45-54 Years Population"
                                       })
del merged_pop['CDC_AgeGroups_x']
del merged_pop['CDC_AgeGroups_y']
del merged_pop['CDC_AgeGroups']
#merged_pop.head()

In [629]:
merged_pop = pd.merge(merged_pop, years55_64_pop, on ='State')
merged_pop = pd.merge(merged_pop, years65_74_pop, on='State')
merged_pop = pd.merge(merged_pop, years75_84_pop, on='State')
merged_pop = merged_pop.rename(columns={
                                "Population_x":"55-64 Years Population",
                                "Population_y":"65-74 Years Population",
                                "Population":"75-84 Years Population"
                                       })
del merged_pop['CDC_AgeGroups_x']
del merged_pop['CDC_AgeGroups_y']
del merged_pop['CDC_AgeGroups']
#merged_pop.head()

In [630]:
merged_pop = pd.merge(merged_pop, years85_pop, on ='State')
merged_pop = pd.merge(merged_pop, all_ages_pop, on ='State')


merged_pop = merged_pop.rename(columns={
                                "Population_x":"85 Years and Over Population",
                                "Population_y":"Total Population"
                                       })

del merged_pop['CDC_AgeGroups_x']
del merged_pop['CDC_AgeGroups_y']
merged_pop

,State,Under 5 Years Population,5-14 Years Population,15-24 Years Population,25-34 Years Population,35-44 Years Population,45-54 Years Population,55-64 Years Population,65-74 Years Population,75-84 Years Population,85 Years and Over Population,Total Population
0,AL,294357.0,608466.0,635707.0,647229.0,593306.0,616616.0,657667.0,501447.0,256847.0,91543.0,4903185.0
1,AK,51080.0,100662.0,96393.0,117588.0,95765.0,85130.0,93339.0,61300.0,23107.0,7181.0,731545.0
2,AZ,429788.0,930745.0,973547.0,1007093.0,896027.0,852090.0,880794.0,751699.0,411197.0,145737.0,7278717.0
3,AR,188464.0,393213.0,397538.0,396868.0,370263.0,361582.0,385994.0,302829.0,161141.0,59912.0,3017804.0
4,CA,2383716.0,5007987.0,5180973.0,6052952.0,5282100.0,4979745.0,4786635.0,3386670.0,1701599.0,749846.0,39512223.0
5,CO,332201.0,710322.0,743213.0,913354.0,798071.0,706150.0,713013.0,522131.0,230360.0,89921.0,5758736.0
6,CT,181710.0,409000.0,478803.0,448029.0,427141.0,476905.0,513455.0,352819.0,186095.0,91330.0,3565287.0
7,DE,54719.0,113975.0,117372.0,129080.0,113101.0,118881.0,137730.0,112858.0,55794.0,20254.0,973764.0
8,FL,1139742.0,2372397.0,2460558.0,2799422.0,2600531.0,2709395.0,2898355.0,2465369.0,1450514.0,581454.0,21477737.0
9,GA,656566.0,1413003.0,1449500.0,1493261.0,1380954.0,1399652.0,1307533.0,926001.0,439884.0,151069.0,10617423.0


In [631]:
merged_pop['Total Population'].sum()

327533774.0

In [632]:
new_row_pop = {'State':'USA', 'Under 5 Years Population':19531315.0, 
              '5-14 Years Population':40926765.0, 
              '15-24 Years Population':42599542.0,
              '25-34 Years Population':45775780.0,
              '35-44 Years Population':41550175.0,
              '45-54 Years Population':40800616.0,
              '55-64 Years Population':42378661.0,
              '65-74 Years Population':31433214.0,
              '75-84 Years Population':15944990.0,
              '85 Years and Over Population':6592716.0,
                'Total Population': 327533774.0
              
             }
#append row to the dataframe
merged_pop = merged_pop.append(new_row_pop, ignore_index=True)

In [633]:
new_dfff = pd.merge(age_sex_df, merged_pop, on='State')

In [634]:
new_dfff = new_dfff[['State', 'Male COVID-19 Deaths', 'Female COVID-19 Deaths', 'All Sexes COVID-19 Deaths',
                     'Under 5 Years COVID-19 Deaths', 'Under 5 Years Population',
                    '5-14 Years COVID-19 Deaths', '5-14 Years Population',
                    '15-24 Years COVID-19 Deaths', '15-24 Years Population',
                     '25-34 Years COVID-19 Deaths', '25-34 Years Population',
                     '35-44 Years COVID-19 Deaths', '35-44 Years Population',
                     '45-54 Years COVID-19 Deaths', '45-54 Years Population',
                     '55-64 Years COVID-19 Deaths', '55-64 Years Population',
                     '65-74 Years COVID-19 Deaths', '65-74 Years Population',
                     '75-84 Years COVID-19 Deaths', '75-84 Years Population',
                     '85 Years and Over COVID-19 Deaths', '85 Years and Over Population',
                     'All Ages COVID-19 Deaths', 'Total Population'
                    ]]
#new_dfff.head()

In [635]:
#new_dfff.to_csv("../data-file/total-pop-death-data.csv", index=False, header=True)

In [636]:
base_url = "https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_002E,B01001_026E&for=state:*"
response = requests.get(base_url)


response = requests.get(base_url)
census_data = response.json()

In [637]:
df = pd.DataFrame(census_data[1:], columns=census_data[0])
df = df.sort_values(by=['state'])

In [638]:
df = df.rename(columns={"B01001_002E":'Male Population', 'B01001_026E':'Female Population'})

In [639]:
df["Male Population"] = pd.to_numeric(df["Male Population"], downcast="float")
df["Female Population"] = pd.to_numeric(df["Female Population"], downcast="float")

In [640]:
new_row = {'NAME':'United States', 'Male Population':163104960.0, 
              'Female Population':168328240.0
             }
#append row to the dataframe
df = df.append(new_row, ignore_index=True)

In [641]:
df = df.drop(columns=['state'])

In [642]:
df = df.drop([8, 51])

In [643]:
df = df.reset_index(drop=True)

In [644]:
for index, row in df.iterrows():
    state = row[0]
    df.loc[index,"NAME"] = us_state_abbrev[state]

In [645]:
df = df.rename(columns={'NAME': 'State'})

In [646]:
pop_merged = pd.merge(df, new_dfff, on = 'State')

In [647]:
pop_merged = pop_merged[['State', 'Male COVID-19 Deaths', 'Male Population', 'Female COVID-19 Deaths', 'Female Population',
                     'All Sexes COVID-19 Deaths', 'Under 5 Years COVID-19 Deaths', 'Under 5 Years Population',
                    '5-14 Years COVID-19 Deaths', '5-14 Years Population',
                    '15-24 Years COVID-19 Deaths', '15-24 Years Population',
                     '25-34 Years COVID-19 Deaths', '25-34 Years Population',
                     '35-44 Years COVID-19 Deaths', '35-44 Years Population',
                     '45-54 Years COVID-19 Deaths', '45-54 Years Population',
                     '55-64 Years COVID-19 Deaths', '55-64 Years Population',
                     '65-74 Years COVID-19 Deaths', '65-74 Years Population',
                     '75-84 Years COVID-19 Deaths', '75-84 Years Population',
                     '85 Years and Over COVID-19 Deaths', '85 Years and Over Population',
                     'All Ages COVID-19 Deaths', 'Total Population'
                    ]]
pop_merged.head()

,State,Male COVID-19 Deaths,Male Population,Female COVID-19 Deaths,Female Population,All Sexes COVID-19 Deaths,Under 5 Years COVID-19 Deaths,Under 5 Years Population,5-14 Years COVID-19 Deaths,5-14 Years Population,...,55-64 Years COVID-19 Deaths,55-64 Years Population,65-74 Years COVID-19 Deaths,65-74 Years Population,75-84 Years COVID-19 Deaths,75-84 Years Population,85 Years and Over COVID-19 Deaths,85 Years and Over Population,All Ages COVID-19 Deaths,Total Population
0,AL,5691,2369611.0,4944,2533574.0,10635,0,294357.0,0,608466.0,...,1482,657667.0,2761,501447.0,3059,256847.0,2440,91543.0,10635,4903185.0
1,AK,197,380433.0,139,351112.0,336,0,51080.0,0,100662.0,...,40,93339.0,86,61300.0,98,23107.0,78,7181.0,336,731545.0
2,AZ,9251,3620935.0,6453,3657782.0,15704,0,429788.0,0,930745.0,...,2290,880794.0,3751,751699.0,4478,411197.0,3536,145737.0,15704,7278717.0
3,AR,3094,1474705.0,2863,1543099.0,5957,0,188464.0,0,393213.0,...,724,385994.0,1341,302829.0,1758,161141.0,1739,59912.0,5957,3017804.0
4,CA,37841,19640794.0,26217,19871428.0,64058,0,2383716.0,0,5007987.0,...,10193,4786635.0,14564,3386670.0,15725,1701599.0,16531,749846.0,64058,39512223.0


In [648]:
sex_df = pop_merged[["State", "Male COVID-19 Deaths", "Male Population", "Female COVID-19 Deaths", "Female Population"]]
sex_df.head()
sex_df["Male COVID-19 Deaths"] = pd.to_numeric(sex_df["Male COVID-19 Deaths"], downcast="float")
sex_df["Female COVID-19 Deaths"] = pd.to_numeric(sex_df["Female COVID-19 Deaths"], downcast="float")
sex_df.head()

<ipython-input-648-8b9566e64ac8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_df["Male COVID-19 Deaths"] = pd.to_numeric(sex_df["Male COVID-19 Deaths"], downcast="float")
<ipython-input-648-8b9566e64ac8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_df["Female COVID-19 Deaths"] = pd.to_numeric(sex_df["Female COVID-19 Deaths"], downcast="float")


,State,Male COVID-19 Deaths,Male Population,Female COVID-19 Deaths,Female Population
0,AL,5691.0,2369611.0,4944.0,2533574.0
1,AK,197.0,380433.0,139.0,351112.0
2,AZ,9251.0,3620935.0,6453.0,3657782.0
3,AR,3094.0,1474705.0,2863.0,1543099.0
4,CA,37841.0,19640794.0,26217.0,19871428.0


In [697]:
sex_df['Crude Male Mortality Rate (%)'] = sex_df['Male COVID-19 Deaths']/sex_df['Male Population']*100
sex_df['Crude Female Mortality Rate (%)'] = sex_df['Female COVID-19 Deaths']/sex_df['Female Population']*100


In [699]:
sex_df = sex_df[['State', 'Male COVID-19 Deaths', 'Male Population', 'Crude Male Mortality Rate (%)', 'Female COVID-19 Deaths', 'Female Population', 'Crude Female Mortality Rate (%)']]

In [701]:
sex_df['total pop'] = sex_df['Male Population'] + sex_df['Female Population']
sex_df.head()

<ipython-input-701-3dd7402544a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_df['total pop'] = sex_df['Male Population'] + sex_df['Female Population']


,State,Male COVID-19 Deaths,Male Population,Crude Male Mortality Rate (%),Female COVID-19 Deaths,Female Population,Crude Female Mortality Rate (%),total pop
0,AL,5691.0,2369611.0,0.240166,4944.0,2533574.0,0.195139,4903185.0
1,AK,197.0,380433.0,0.051783,139.0,351112.0,0.039589,731545.0
2,AZ,9251.0,3620935.0,0.255486,6453.0,3657782.0,0.176418,7278717.0
3,AR,3094.0,1474705.0,0.209805,2863.0,1543099.0,0.185536,3017804.0
4,CA,37841.0,19640794.0,0.192665,26217.0,19871428.0,0.131933,39512222.0


In [703]:
sex_df['Male Population (%)'] = sex_df['Male Population']/sex_df['total pop'] *100
sex_df['Female Population (%)'] = sex_df['Female Population']/sex_df['total pop']*100
sex_df.head()

,State,Male COVID-19 Deaths,Male Population,Crude Male Mortality Rate (%),Female COVID-19 Deaths,Female Population,Crude Female Mortality Rate (%),total pop,Male Population (%),Female Population (%)
0,AL,5691.0,2369611.0,0.240166,4944.0,2533574.0,0.195139,4903185.0,48.327995,51.672005
1,AK,197.0,380433.0,0.051783,139.0,351112.0,0.039589,731545.0,52.004046,47.995954
2,AZ,9251.0,3620935.0,0.255486,6453.0,3657782.0,0.176418,7278717.0,49.746885,50.253115
3,AR,3094.0,1474705.0,0.209805,2863.0,1543099.0,0.185536,3017804.0,48.866825,51.133175
4,CA,37841.0,19640794.0,0.192665,26217.0,19871428.0,0.131933,39512222.0,49.708149,50.291851


In [704]:
sex_df = sex_df[['State', 'Male Population (%)', 'Crude Male Mortality Rate (%)', 'Female Population (%)', 'Crude Female Mortality Rate (%)']]
sex_df.head()

,State,Male Population (%),Crude Male Mortality Rate (%),Female Population (%),Crude Female Mortality Rate (%)
0,AL,48.327995,0.240166,51.672005,0.195139
1,AK,52.004046,0.051783,47.995954,0.039589
2,AZ,49.746885,0.255486,50.253115,0.176418
3,AR,48.866825,0.209805,51.133175,0.185536
4,CA,49.708149,0.192665,50.291851,0.131933


In [705]:
sex_df.to_csv("../jennie-folder/demographic_sex.csv", index=False, header=True)

In [653]:
race_url = "https://data.cdc.gov/resource/pj7m-y5uh.json"
response = requests.get(race_url)
race_data = response.json()

In [654]:
client = Socrata("data.cdc.gov", None)

race_results = client.get("pj7m-y5uh", limit=6000)

race_df = pd.DataFrame.from_records(race_results)

In [655]:
race_df = race_df.loc[race_df['group']=="By Total"]
race_df = race_df.loc[race_df['indicator']=="Count of COVID-19 deaths"]

In [656]:
race_df = race_df.drop(columns=['data_as_of', 'start_week', 'end_week', 'year', 'group', 'indicator', 'month', 'footnote'])

In [657]:
race_df = race_df.rename(columns={'state':'State', 'non_hispanic_white': "White Deaths",
                                  'non_hispanic_black_african_american':'Black/African American Deaths',
                                 'non_hispanic_american_indian_alaska_native':'American Indian/Alaska Native Deaths',
                                  'non_hispanic_asian_pacific_islander':'Asian Pacific Islander Deaths',
                                  'nh_nhopi':'Native Hawaiian/Other Pacific Islander Deaths',
                                  'non_hispanic_more_than_one_race':'More than one Race Deaths',
                                  'hispanic_latino_total':'Hispanic/Latino Deaths'
                                 })
#race_df.head()

In [658]:
race_df = race_df.reset_index(drop=True)

In [659]:
for index, row in race_df.iterrows():
    state = row[0]
    race_df.loc[index,"State"] = us_state_abbrev[state]

In [660]:
race_df = race_df.fillna(0)

In [661]:
#race_df.head()

In [662]:
race_pop_url = "https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001A_001E,B01001B_001E,B01001C_001E,B01001D_001E,B01001E_001E,B01001G_002E,B01001I_001E&for=state:*"
response = requests.get(race_pop_url)


race_pop_response = requests.get(race_pop_url)
race_pop_data = race_pop_response.json()

In [663]:
race_pop_df = pd.DataFrame(race_pop_data[1:], columns=race_pop_data[0])
race_pop_df = race_pop_df.sort_values(by=['state'])

In [664]:
race_pop_df = race_pop_df.reset_index(drop=True)

In [665]:
race_pop_df = race_pop_df.rename(columns={'NAME':'State','B01001A_001E':'White Population',
                                          'B01001B_001E':'Black/African American Population',
                                          'B01001C_001E':'American Indian/Alaska Native Population',
                                          'B01001D_001E':'Asian Population',
                                          'B01001E_001E':'Native Hawaiian/Other Pacific Islander Population',
                                          'B01001G_002E':'More than one Race Population',
                                          'B01001I_001E':'Hispanic/Latino Population'
                                         })
#race_pop_df.head()

In [666]:
race_pop_df = race_pop_df.fillna(0)

In [667]:
race_pop_df["White Population"] = pd.to_numeric(race_pop_df["White Population"], downcast="float")
race_pop_df["Black/African American Population"] = pd.to_numeric(race_pop_df["Black/African American Population"], downcast="float")
race_pop_df["American Indian/Alaska Native Population"] = pd.to_numeric(race_pop_df["American Indian/Alaska Native Population"], downcast="float")
race_pop_df["Asian Population"] = pd.to_numeric(race_pop_df["Asian Population"], downcast="float")
race_pop_df["Native Hawaiian/Other Pacific Islander Population"] = pd.to_numeric(race_pop_df["Native Hawaiian/Other Pacific Islander Population"], downcast="float")
race_pop_df["More than one Race Population"] = pd.to_numeric(race_pop_df["More than one Race Population"], downcast="float")
race_pop_df["Hispanic/Latino Population"] = pd.to_numeric(race_pop_df["Hispanic/Latino Population"], downcast="float")

In [668]:
new_row_race = {'State':'United States', 'White Population':238530160.0, 
              'Black/African American Population':42357500.0,
               'American Indian/Alaska Native Population':2815045.0,
               'Asian Population':18632352.0,
               'Native Hawaiian/Other Pacific Islander Population':496290.0,
               'More than one Race Population':5752848.0,
               'Hispanic/Latino Population':63616508.0
             }
#append row to the dataframe
race_pop_df = race_pop_df.append(new_row_race, ignore_index=True)

In [669]:
race_pop_df = race_pop_df.drop(columns=['state'])

In [670]:
race_pop_df = race_pop_df.drop([8, 51])

In [671]:
race_pop_df = race_pop_df.reset_index(drop=True)

In [672]:
for index, row in race_pop_df.iterrows():
    state = row[0]
    race_pop_df.loc[index,"State"] = us_state_abbrev[state]

In [673]:
race_pop_df.head()

,State,White Population,Black/African American Population,American Indian/Alaska Native Population,Asian Population,Native Hawaiian/Other Pacific Islander Population,More than one Race Population,Hispanic/Latino Population
0,AL,3326375.0,1319551.0,23265.0,66129.0,0.0,42835.0,219296.0
1,AK,469771.0,22551.0,115544.0,43678.0,0.0,28459.0,52548.0
2,AZ,5701810.0,343729.0,332273.0,241721.0,14168.0,146550.0,2310592.0
3,AR,2315020.0,467468.0,17216.0,46078.0,12829.0,42358.0,231951.0
4,CA,23484958.0,2282144.0,321112.0,5865435.0,155871.0,999761.0,15574882.0


In [674]:
race_merge = pd.merge(race_df, race_pop_df, on ='State')

In [675]:
race_merge = race_merge[['State', 
                         'White Deaths', 'White Population', 
                         'Black/African American Deaths', 'Black/African American Population',
                         'American Indian/Alaska Native Deaths', 'American Indian/Alaska Native Population', 
                         'Asian Pacific Islander Deaths', 'Asian Population',
                         'Native Hawaiian/Other Pacific Islander Deaths', 'Native Hawaiian/Other Pacific Islander Population',
                         'More than one Race Deaths', 'More than one Race Population',
                         'Hispanic/Latino Deaths', 'Hispanic/Latino Population'
                    ]]
race_merge.head()

,State,White Deaths,White Population,Black/African American Deaths,Black/African American Population,American Indian/Alaska Native Deaths,American Indian/Alaska Native Population,Asian Pacific Islander Deaths,Asian Population,Native Hawaiian/Other Pacific Islander Deaths,Native Hawaiian/Other Pacific Islander Population,More than one Race Deaths,More than one Race Population,Hispanic/Latino Deaths,Hispanic/Latino Population
0,USA,348558,238530160.0,85495,42357500.0,6482,2815045.0,22190,18632352.0,1044,496290.0,1864,5752848.0,106465,63616508.0
1,AL,7231,3326375.0,3130,1319551.0,17,23265.0,48,66129.0,0,0.0,0,42835.0,190,219296.0
2,AK,133,469771.0,0,22551.0,114,115544.0,34,43678.0,20,0.0,10,28459.0,15,52548.0
3,AZ,8084,5701810.0,507,343729.0,1605,332273.0,325,241721.0,36,14168.0,143,146550.0,4849,2310592.0
4,AR,4671,2315020.0,883,467468.0,39,17216.0,47,46078.0,58,12829.0,24,42358.0,235,231951.0


In [676]:
demographic_age = pd.merge(pop_merged, race_merge, how='left', on='State')
demographic_age.head()

,State,Male COVID-19 Deaths,Male Population,Female COVID-19 Deaths,Female Population,All Sexes COVID-19 Deaths,Under 5 Years COVID-19 Deaths,Under 5 Years Population,5-14 Years COVID-19 Deaths,5-14 Years Population,...,American Indian/Alaska Native Deaths,American Indian/Alaska Native Population,Asian Pacific Islander Deaths,Asian Population,Native Hawaiian/Other Pacific Islander Deaths,Native Hawaiian/Other Pacific Islander Population,More than one Race Deaths,More than one Race Population,Hispanic/Latino Deaths,Hispanic/Latino Population
0,AL,5691,2369611.0,4944,2533574.0,10635,0,294357.0,0,608466.0,...,17,23265.0,48,66129.0,0,0.0,0,42835.0,190,219296.0
1,AK,197,380433.0,139,351112.0,336,0,51080.0,0,100662.0,...,114,115544.0,34,43678.0,20,0.0,10,28459.0,15,52548.0
2,AZ,9251,3620935.0,6453,3657782.0,15704,0,429788.0,0,930745.0,...,1605,332273.0,325,241721.0,36,14168.0,143,146550.0,4849,2310592.0
3,AR,3094,1474705.0,2863,1543099.0,5957,0,188464.0,0,393213.0,...,39,17216.0,47,46078.0,58,12829.0,24,42358.0,235,231951.0
4,CA,37841,19640794.0,26217,19871428.0,64058,0,2383716.0,0,5007987.0,...,297,321112.0,7994,5865435.0,350,155871.0,397,999761.0,30574,15574882.0


In [677]:
demographic_df = pd.read_csv("../jennie-folder/CRDT Data - CRDT.csv")

In [678]:
demographic_df.head()

,Date,State,Cases_Total,Cases_White,Cases_Black,Cases_Latinx,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,...,Tests_Latinx,Tests_Asian,Tests_AIAN,Tests_NHPI,Tests_Multiracial,Tests_Other,Tests_Unknown,Tests_Ethnicity_Hispanic,Tests_Ethnicity_NonHispanic,Tests_Ethnicity_Unknown
0,20210307,AK,59332.0,18300.0,1499.0,NaN,2447.0,12238.0,1508.0,4453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210307,AL,499819.0,160347.0,82790.0,NaN,2273.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210307,AR,324818.0,207596.0,50842.0,NaN,2913.0,1070.0,3358.0,1804.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210307,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210307,AZ,826454.0,308453.0,25775.0,244539.0,11921.0,40707.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [679]:
print(demographic_df.columns.tolist())

['Date', 'State', 'Cases_Total', 'Cases_White', 'Cases_Black', 'Cases_Latinx', 'Cases_Asian', 'Cases_AIAN', 'Cases_NHPI', 'Cases_Multiracial', 'Cases_Other', 'Cases_Unknown', 'Cases_Ethnicity_Hispanic', 'Cases_Ethnicity_NonHispanic', 'Cases_Ethnicity_Unknown', 'Deaths_Total', 'Deaths_White', 'Deaths_Black', 'Deaths_Latinx', 'Deaths_Asian', 'Deaths_AIAN', 'Deaths_NHPI', 'Deaths_Multiracial', 'Deaths_Other', 'Deaths_Unknown', 'Deaths_Ethnicity_Hispanic', 'Deaths_Ethnicity_NonHispanic', 'Deaths_Ethnicity_Unknown', 'Hosp_Total', 'Hosp_White', 'Hosp_Black', 'Hosp_Latinx', 'Hosp_Asian', 'Hosp_AIAN', 'Hosp_NHPI', 'Hosp_Multiracial', 'Hosp_Other', 'Hosp_Unknown', 'Hosp_Ethnicity_Hispanic', 'Hosp_Ethnicity_NonHispanic', 'Hosp_Ethnicity_Unknown', 'Tests_Total', 'Tests_White', 'Tests_Black', 'Tests_Latinx', 'Tests_Asian', 'Tests_AIAN', 'Tests_NHPI', 'Tests_Multiracial', 'Tests_Other', 'Tests_Unknown', 'Tests_Ethnicity_Hispanic', 'Tests_Ethnicity_NonHispanic', 'Tests_Ethnicity_Unknown']


In [680]:
demographic_df = demographic_df.groupby('State').sum()
demographic_df = demographic_df[["Cases_Total", "Cases_White", "Cases_Black","Cases_Latinx", "Cases_Asian", "Cases_AIAN", "Cases_NHPI", "Cases_Multiracial", "Cases_Ethnicity_Hispanic"]]
demographic_df.head()

,Cases_Total,Cases_White,Cases_Black,Cases_Latinx,Cases_Asian,Cases_AIAN,Cases_NHPI,Cases_Multiracial,Cases_Ethnicity_Hispanic
State,,,,,,,,,
AK,1855334.0,574849.0,56782.0,0.0,73005.0,373509.0,54235.0,127827.0,106683.0
AL,17861341.0,5601494.0,3373720.0,0.0,70420.0,0.0,0.0,0.0,651897.0
AR,10837342.0,6652666.0,1890196.0,0.0,105929.0,31859.0,195756.0,61984.0,1452815.0
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZ,27502303.0,8458144.0,815793.0,7948222.0,332233.0,1458070.0,0.0,0.0,7947570.0


In [681]:
death_rate_eth = pd.merge(race_merge, demographic_df, on='State')

In [682]:
death_rate_eth = death_rate_eth[['State',
                         'Cases_White','White Deaths', 'White Population', 
                         'Cases_Black', 'Black/African American Deaths', 'Black/African American Population',
                         'Cases_AIAN','American Indian/Alaska Native Deaths', 'American Indian/Alaska Native Population', 
                         'Cases_Asian','Asian Pacific Islander Deaths', 'Asian Population',
                         'Cases_NHPI','Native Hawaiian/Other Pacific Islander Deaths', 'Native Hawaiian/Other Pacific Islander Population',
                         'Cases_Multiracial', 'More than one Race Deaths', 'More than one Race Population',
                         'Cases_Ethnicity_Hispanic','Hispanic/Latino Deaths', 'Hispanic/Latino Population'
                    ]]
death_rate_eth.head()

death_rate_eth['total pop'] = death_rate_eth['White Population'] + death_rate_eth['Black/African American Population'] + death_rate_eth['American Indian/Alaska Native Population'] + death_rate_eth['Asian Population'] + death_rate_eth['Native Hawaiian/Other Pacific Islander Population'] + death_rate_eth['More than one Race Population'] + death_rate_eth['Hispanic/Latino Population']
death_rate_eth.head()

,State,Cases_White,White Deaths,White Population,Cases_Black,Black/African American Deaths,Black/African American Population,Cases_AIAN,American Indian/Alaska Native Deaths,American Indian/Alaska Native Population,...,Cases_NHPI,Native Hawaiian/Other Pacific Islander Deaths,Native Hawaiian/Other Pacific Islander Population,Cases_Multiracial,More than one Race Deaths,More than one Race Population,Cases_Ethnicity_Hispanic,Hispanic/Latino Deaths,Hispanic/Latino Population,total pop
0,AL,5601494.0,7231,3326375.0,3373720.0,3130,1319551.0,0.0,17,23265.0,...,0.0,0,0.0,0.0,0,42835.0,651897.0,190,219296.0,4997451.0
1,AK,574849.0,133,469771.0,56782.0,0,22551.0,373509.0,114,115544.0,...,54235.0,20,0.0,127827.0,10,28459.0,106683.0,15,52548.0,732551.0
2,AZ,8458144.0,8084,5701810.0,815793.0,507,343729.0,1458070.0,1605,332273.0,...,0.0,36,14168.0,0.0,143,146550.0,7947570.0,4849,2310592.0,9090843.0
3,AR,6652666.0,4671,2315020.0,1890196.0,883,467468.0,31859.0,39,17216.0,...,195756.0,58,12829.0,61984.0,24,42358.0,1452815.0,235,231951.0,3132920.0
4,CA,15360536.0,20339,23484958.0,3408823.0,3988,2282144.0,241503.0,297,321112.0,...,448957.0,350,155871.0,1022184.0,397,999761.0,45105227.0,30574,15574882.0,48684163.0


In [683]:
death_rate_eth["White Deaths"] = pd.to_numeric(death_rate_eth["White Deaths"], downcast="float")
death_rate_eth["Black/African American Deaths"] = pd.to_numeric(death_rate_eth["Black/African American Deaths"], downcast="float")
death_rate_eth["American Indian/Alaska Native Deaths"] = pd.to_numeric(death_rate_eth["American Indian/Alaska Native Deaths"], downcast="float")
death_rate_eth["Native Hawaiian/Other Pacific Islander Deaths"] = pd.to_numeric(death_rate_eth["Native Hawaiian/Other Pacific Islander Deaths"], downcast="float")
death_rate_eth["Asian Pacific Islander Deaths"] = pd.to_numeric(death_rate_eth["Asian Pacific Islander Deaths"], downcast="float")
death_rate_eth["More than one Race Deaths"] = pd.to_numeric(death_rate_eth["More than one Race Deaths"], downcast="float")
death_rate_eth["Hispanic/Latino Deaths"] = pd.to_numeric(death_rate_eth["Hispanic/Latino Deaths"], downcast="float")

In [684]:
white_dr = death_rate_eth['White Deaths']/death_rate_eth['Cases_White']*100
death_rate_eth['White Fatality Rate (%)'] = white_dr

white_pop = death_rate_eth['White Population']/death_rate_eth['total pop']*100
death_rate_eth['White Population %'] = white_pop

black_dr = death_rate_eth['Black/African American Deaths']/death_rate_eth['Cases_Black']*100
death_rate_eth['Black Fatality Rate (%)'] = black_dr

black_pop = death_rate_eth['Black/African American Population']/death_rate_eth['total pop']*100
death_rate_eth['Black/African American Population %'] = black_pop

AIAN_dr = death_rate_eth['American Indian/Alaska Native Deaths']/death_rate_eth['Cases_AIAN']*100
death_rate_eth['AIAN Fatality Rate (%)'] = AIAN_dr

AIAN_pop = death_rate_eth['American Indian/Alaska Native Population']/death_rate_eth['total pop']*100
death_rate_eth['AIAN Population %'] = AIAN_pop

ASIAN_dr = death_rate_eth['Asian Pacific Islander Deaths']/death_rate_eth['Cases_Asian']*100
death_rate_eth['Asian Fatality Rate (%)'] = ASIAN_dr

asian_pop = death_rate_eth['Asian Population']/death_rate_eth['total pop']*100
death_rate_eth['Asian Population %'] = asian_pop

NHPI_dr = death_rate_eth['Native Hawaiian/Other Pacific Islander Deaths']/death_rate_eth['Cases_NHPI']*100
death_rate_eth['NHPI Fatality Rate (%)'] = NHPI_dr

NHPI_pop = death_rate_eth['Native Hawaiian/Other Pacific Islander Population']/death_rate_eth['total pop']*100
death_rate_eth['NHPI Population %'] = NHPI_pop

multiple_dr = death_rate_eth['More than one Race Deaths']/death_rate_eth['Cases_Multiracial']*100
death_rate_eth['Multiracial Fatality Rate (%)'] = multiple_dr

multiple_pop = death_rate_eth['More than one Race Population']/death_rate_eth['total pop']*100
death_rate_eth['Multiracial Population %'] = multiple_pop

hispanic_dr = death_rate_eth['Hispanic/Latino Deaths']/death_rate_eth['Cases_Ethnicity_Hispanic']*100
death_rate_eth['Hispanic Fatality Rate (%)'] = hispanic_dr

hispanic_pop = death_rate_eth['Hispanic/Latino Population']/death_rate_eth['total pop']*100
death_rate_eth['Hispanic Population %'] = hispanic_pop

In [685]:
death_rate_eth.head()

,State,Cases_White,White Deaths,White Population,Cases_Black,Black/African American Deaths,Black/African American Population,Cases_AIAN,American Indian/Alaska Native Deaths,American Indian/Alaska Native Population,...,AIAN Fatality Rate (%),AIAN Population %,Asian Fatality Rate (%),Asian Population %,NHPI Fatality Rate (%),NHPI Population %,Multiracial Fatality Rate (%),Multiracial Population %,Hispanic Fatality Rate (%),Hispanic Population %
0,AL,5601494.0,7231.0,3326375.0,3373720.0,3130.0,1319551.0,0.0,17.0,23265.0,...,inf,0.465537,0.068162,1.323255,NaN,0.000000,NaN,0.857137,0.029146,4.388157
1,AK,574849.0,133.0,469771.0,56782.0,0.0,22551.0,373509.0,114.0,115544.0,...,0.030521,15.772827,0.046572,5.962452,0.036877,0.000000,0.007823,3.884917,0.014060,7.173289
2,AZ,8458144.0,8084.0,5701810.0,815793.0,507.0,343729.0,1458070.0,1605.0,332273.0,...,0.110077,3.655030,0.097823,2.658950,inf,0.155849,inf,1.612062,0.061012,25.416697
3,AR,6652666.0,4671.0,2315020.0,1890196.0,883.0,467468.0,31859.0,39.0,17216.0,...,0.122414,0.549519,0.044369,1.470768,0.029629,0.409490,0.038720,1.352029,0.016175,7.403668
4,CA,15360536.0,20339.0,23484958.0,3408823.0,3988.0,2282144.0,241503.0,297.0,321112.0,...,0.122980,0.659582,0.158423,12.047932,0.077958,0.320168,0.038838,2.053565,0.067784,31.991681


In [686]:
death_rate_eth = death_rate_eth[['State',
                         'White Fatality Rate (%)', 'White Population %',
                         'Black Fatality Rate (%)', 'Black/African American Population %',
                         'AIAN Fatality Rate (%)', 'AIAN Population %',
                         'Asian Fatality Rate (%)', 'Asian Population %',
                         'NHPI Fatality Rate (%)', 'NHPI Population %',
                         'Multiracial Fatality Rate (%)', 'Multiracial Population %',
                         'Hispanic Fatality Rate (%)', 'Hispanic Population %'
                    ]]

death_rate_eth = death_rate_eth.replace([np.inf, -np.inf], np.nan)
death_rate_eth = death_rate_eth.fillna(0)
death_rate_eth.head()

,State,White Fatality Rate (%),White Population %,Black Fatality Rate (%),Black/African American Population %,AIAN Fatality Rate (%),AIAN Population %,Asian Fatality Rate (%),Asian Population %,NHPI Fatality Rate (%),NHPI Population %,Multiracial Fatality Rate (%),Multiracial Population %,Hispanic Fatality Rate (%),Hispanic Population %
0,AL,0.129091,66.561433,0.092776,26.404481,0.000000,0.465537,0.068162,1.323255,0.000000,0.000000,0.000000,0.857137,0.029146,4.388157
1,AK,0.023137,64.128095,0.000000,3.078420,0.030521,15.772827,0.046572,5.962452,0.036877,0.000000,0.007823,3.884917,0.014060,7.173289
2,AZ,0.095577,62.720366,0.062148,3.781046,0.110077,3.655030,0.097823,2.658950,0.000000,0.155849,0.000000,1.612062,0.061012,25.416697
3,AR,0.070212,73.893365,0.046715,14.921160,0.122414,0.549519,0.044369,1.470768,0.029629,0.409490,0.038720,1.352029,0.016175,7.403668
4,CA,0.132411,48.239420,0.116991,4.687652,0.122980,0.659582,0.158423,12.047932,0.077958,0.320168,0.038838,2.053565,0.067784,31.991681


In [687]:
death_rate_eth.to_csv("../jennie-folder/demographic_ethnicity.csv", index=False, header=True)

In [706]:
demographic_age.head()

,State,Male COVID-19 Deaths,Male Population,Female COVID-19 Deaths,Female Population,All Sexes COVID-19 Deaths,Under 5 Years COVID-19 Deaths,Under 5 Years Population,5-14 Years COVID-19 Deaths,5-14 Years Population,...,American Indian/Alaska Native Deaths,American Indian/Alaska Native Population,Asian Pacific Islander Deaths,Asian Population,Native Hawaiian/Other Pacific Islander Deaths,Native Hawaiian/Other Pacific Islander Population,More than one Race Deaths,More than one Race Population,Hispanic/Latino Deaths,Hispanic/Latino Population
0,AL,5691,2369611.0,4944,2533574.0,10635,0,294357.0,0,608466.0,...,17,23265.0,48,66129.0,0,0.0,0,42835.0,190,219296.0
1,AK,197,380433.0,139,351112.0,336,0,51080.0,0,100662.0,...,114,115544.0,34,43678.0,20,0.0,10,28459.0,15,52548.0
2,AZ,9251,3620935.0,6453,3657782.0,15704,0,429788.0,0,930745.0,...,1605,332273.0,325,241721.0,36,14168.0,143,146550.0,4849,2310592.0
3,AR,3094,1474705.0,2863,1543099.0,5957,0,188464.0,0,393213.0,...,39,17216.0,47,46078.0,58,12829.0,24,42358.0,235,231951.0
4,CA,37841,19640794.0,26217,19871428.0,64058,0,2383716.0,0,5007987.0,...,297,321112.0,7994,5865435.0,350,155871.0,397,999761.0,30574,15574882.0


In [688]:
aged_df = demographic_age[["State", "65-74 Years Population", "65-74 Years COVID-19 Deaths", "75-84 Years Population","75-84 Years COVID-19 Deaths", "85 Years and Over Population", "85 Years and Over COVID-19 Deaths"]]
aged_df.head()

,State,65-74 Years Population,65-74 Years COVID-19 Deaths,75-84 Years Population,75-84 Years COVID-19 Deaths,85 Years and Over Population,85 Years and Over COVID-19 Deaths
0,AL,501447.0,2761,256847.0,3059,91543.0,2440
1,AK,61300.0,86,23107.0,98,7181.0,78
2,AZ,751699.0,3751,411197.0,4478,145737.0,3536
3,AR,302829.0,1341,161141.0,1758,59912.0,1739
4,CA,3386670.0,14564,1701599.0,15725,749846.0,16531


In [689]:
aged_df["65-74 Years COVID-19 Deaths"] = aged_df["65-74 Years COVID-19 Deaths"].astype(float)
aged_df["75-84 Years COVID-19 Deaths"] = aged_df["75-84 Years COVID-19 Deaths"].astype(float)
aged_df["85 Years and Over COVID-19 Deaths"] = aged_df["85 Years and Over COVID-19 Deaths"].astype(float)

<ipython-input-689-2b0fc69b8258>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df["65-74 Years COVID-19 Deaths"] = aged_df["65-74 Years COVID-19 Deaths"].astype(float)
<ipython-input-689-2b0fc69b8258>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df["75-84 Years COVID-19 Deaths"] = aged_df["75-84 Years COVID-19 Deaths"].astype(float)
<ipython-input-689-2b0fc69b8258>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [690]:
aged_df['65 Years and Over Population'] = aged_df['65-74 Years Population'] + aged_df['75-84 Years Population'] + aged_df['85 Years and Over Population']
aged_df.head()

<ipython-input-690-ba8fd7fc5e01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df['65 Years and Over Population'] = aged_df['65-74 Years Population'] + aged_df['75-84 Years Population'] + aged_df['85 Years and Over Population']


,State,65-74 Years Population,65-74 Years COVID-19 Deaths,75-84 Years Population,75-84 Years COVID-19 Deaths,85 Years and Over Population,85 Years and Over COVID-19 Deaths,65 Years and Over Population
0,AL,501447.0,2761.0,256847.0,3059.0,91543.0,2440.0,849837.0
1,AK,61300.0,86.0,23107.0,98.0,7181.0,78.0,91588.0
2,AZ,751699.0,3751.0,411197.0,4478.0,145737.0,3536.0,1308633.0
3,AR,302829.0,1341.0,161141.0,1758.0,59912.0,1739.0,523882.0
4,CA,3386670.0,14564.0,1701599.0,15725.0,749846.0,16531.0,5838115.0


In [691]:
aged_df['65 Years and Over COVID Deaths'] = aged_df['65-74 Years COVID-19 Deaths'] + aged_df['75-84 Years COVID-19 Deaths'] + aged_df['85 Years and Over COVID-19 Deaths']
aged_df.head()

<ipython-input-691-cae404275036>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df['65 Years and Over COVID Deaths'] = aged_df['65-74 Years COVID-19 Deaths'] + aged_df['75-84 Years COVID-19 Deaths'] + aged_df['85 Years and Over COVID-19 Deaths']


,State,65-74 Years Population,65-74 Years COVID-19 Deaths,75-84 Years Population,75-84 Years COVID-19 Deaths,85 Years and Over Population,85 Years and Over COVID-19 Deaths,65 Years and Over Population,65 Years and Over COVID Deaths
0,AL,501447.0,2761.0,256847.0,3059.0,91543.0,2440.0,849837.0,8260.0
1,AK,61300.0,86.0,23107.0,98.0,7181.0,78.0,91588.0,262.0
2,AZ,751699.0,3751.0,411197.0,4478.0,145737.0,3536.0,1308633.0,11765.0
3,AR,302829.0,1341.0,161141.0,1758.0,59912.0,1739.0,523882.0,4838.0
4,CA,3386670.0,14564.0,1701599.0,15725.0,749846.0,16531.0,5838115.0,46820.0


In [692]:
population_over = (aged_df['65 Years and Over Population']/53970920.0)*10
aged_df["Population % for 65 Years and Over"] = population_over

<ipython-input-692-84d930033a4e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df["Population % for 65 Years and Over"] = population_over


In [693]:
deaths_data = (aged_df['65 Years and Over COVID Deaths']/aged_df['65 Years and Over Population'] * 100)
aged_df["65 Years and Over Crude Mortality Rate (%)"] = deaths_data

<ipython-input-693-7be2f73fb42b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aged_df["65 Years and Over Crude Mortality Rate (%)"] = deaths_data


In [694]:
aged_df = aged_df[["State", "65 Years and Over Population", "65 Years and Over COVID Deaths", "65 Years and Over Crude Mortality Rate (%)" ]]
aged_df

,State,65 Years and Over Population,65 Years and Over COVID Deaths,65 Years and Over Crude Mortality Rate (%)
0,AL,849837.0,8260.0,0.971951
1,AK,91588.0,262.0,0.286064
2,AZ,1308633.0,11765.0,0.899030
3,AR,523882.0,4838.0,0.923490
4,CA,5838115.0,46820.0,0.801971
5,CO,842412.0,5482.0,0.650750
6,CT,630244.0,7015.0,1.113061
7,DE,188906.0,1230.0,0.651117
8,FL,4497337.0,27296.0,0.606937
9,GA,1516954.0,13275.0,0.875109


In [695]:
aged_df.to_csv("../jennie-folder/demographic_age.csv", index=False, header=True)

In [696]:
aged_df.style.format({
    '65 Years and Over Population': '{:,.2f}'.format,
    'Population % for 65 Years and Over': '{:,.2f}%'.format,
    '65 Years and Over COVID Deaths': '{:,.2f}'.format,
    '65 Years and Over Crude Mortality Rate': '{:,.2f}%'.format
})

,State,65 Years and Over Population,65 Years and Over COVID Deaths,65 Years and Over Crude Mortality Rate (%)
0,AL,"849,837.00","8,260.00",0.971951
1,AK,"91,588.00",262.00,0.286064
2,AZ,"1,308,633.00","11,765.00",0.899030
3,AR,"523,882.00","4,838.00",0.923490
4,CA,"5,838,115.00","46,820.00",0.801971
5,CO,"842,412.00","5,482.00",0.650750
6,CT,"630,244.00","7,015.00",1.113061
7,DE,"188,906.00","1,230.00",0.651117
8,FL,"4,497,337.00","27,296.00",0.606937
9,GA,"1,516,954.00","13,275.00",0.875109
